In [ ]:
from google.colab import files
files.upload()


In [ ]:
!ls -lha kaggle.json

In [ ]:
!pip install -q kaggle

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
import pandas as pd

In [ ]:
!unzip /content/icml_face_data.csv.zip

In [ ]:
df = pd.read_csv('icml_face_data.csv')

In [ ]:
df.head()

In [ ]:
data = df.values

In [ ]:
y = data[:, 0]
pixels = data[:, 2]

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Convolution2D, UpSampling2D, MaxPooling2D, ZeroPadding2D, Flatten, Dropout, Reshape
from keras.models import Sequential
from keras.utils import np_utils

In [ ]:
X = np.zeros((pixels.shape[0], 48*48))
# Делаем из одномерного массива двумерный, чтобы сформировать фотографии датасета
for ix in range(X.shape[0]):
    p = pixels[ix].split(' ')
    for iy in range(X.shape[1]):
        X[ix, iy] = int(p[iy])

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Смотрим что в датасете
for ix in range(4):
    plt.figure(ix)
    plt.imshow(X[ix].reshape((48, 48)), interpolation='none', cmap='gray')
plt.show()

In [ ]:
x = X

In [ ]:
# Разбиваем датасет на обучающую 
# и валидационную выборку
X_train = x[0:28710, :]
Y_train = y[0:28710]
X_crossval = x[28710:32300, :]
Y_crossval = y[28710:32300]


In [ ]:
# Делаем приводим выборки к входному формату нейронной сети
X_train = X_train.reshape((X_train.shape[0], 48 , 48, 1 ))
X_crossval = X_crossval.reshape((X_crossval.shape[0], 48 ,48, 1))

In [ ]:
# Выходных классов 7. На это и указываем
y_ = np_utils.to_categorical(y, 7)

In [ ]:
Y_train = y_[:28710]
Y_crossval = y_[28710:32300]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# Генерируем дополнительные данные на основе текущих
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.0,  
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False, 
        vertical_flip=False)  

datagen.fit(X_train)

In [ ]:
# Будем замедлять скорость обучения если прироста в обучении нет

from keras.callbacks import ReduceLROnPlateau
lr_reduce = ReduceLROnPlateau(monitor='val_accurency', factor=0.1, epsilon=0.0001, patience=1, verbose=1)

In [ ]:

from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , Adam
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.common.set_image_dim_ordering('th')

In [ ]:
# Swith-функция активации (Сигмоид(х)*х)
def swish_activation(x):
    return (K.sigmoid(x) * x)
# Модель CNN
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=(48,48,1)))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation=swish_activation))

model.add(Dropout(0.4))

model.add(Dense(7 , activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam' ,
              metrics=['accuracy'])

In [ ]:
# Смотрим как выглядит модель
print(model.summary())

In [ ]:
# Перемешиваем данные разбиваем на батчи
# Задаем оптимизаторы, метрики и кол-во эпох
batch_size = 128
epochs = 14

model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['accuracy'])
steps_per_epoch = len(X) // batch_size
validation_steps = len((X_crossval, Y_crossval)) // batch_size

In [ ]:
# Обучаем модель
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=X.shape[0] // batch_size,
                    callbacks=[lr_reduce],
                    validation_data=(X_crossval, Y_crossval),
                    epochs = epochs, verbose = 2)

In [ ]:
# Графики точности
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Графики потерь
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Предсказываем данные на тесте
prediction = model.predict(X_crossval)

In [ ]:
import cv2
face_path = r'/content/1.jpg'
face = cv2.imread(face_path, 0)
emotion = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
f = face.reshape(1, 48, 48, 1)

In [ ]:
predict = model.predict(f)

In [ ]:
emotion = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
plt.imshow(face.reshape((48, 48)), interpolation='none', cmap='gray')

In [ ]:
emotion[np.argmax(predict)]

In [ ]:
predict[0]

In [ ]:
from google.colab import files
files.upload()

In [ ]:
model.save('emotional_model.h5')

In [ ]:
names = np.arange(7)
plt.bar(names, height=predict[0])
plt.xticks(names, ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'])
plt.show()

In [ ]:
from keras.models import load_model

In [ ]:
def swish_activation(x):
    return (K.sigmoid(x) * x)

In [ ]:
reloaded = load_model('emotional_model.h5', custom_objects={'swish_activation': swish_activation})

In [ ]:
reloaded.summary()

In [ ]:
np.argmax(reloaded.predict(f))